# Open-Ollama-RAG-ChatApp Notebook

## Setup

In [20]:
# %pip install langchain
# %pip install chromadb
# %pip install langchain_community
# %pip install gradio

Note: you may need to restart the kernel to use updated packages.
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.1.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached chroma_hnswlib-0.7.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata (252 bytes)
  Using cached posthog-3.4.2-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached pulsar_client-3.4.0-cp311-cp311-macosx_10_15_universal2.whl.metadata (1.0 kB)
  Using cached onnxruntime-1.16.3-cp311-cp311-macosx_10_15_x86_64.whl.metadata (4.3 kB)
  Using cached opentelemetry_api-1.23.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.44b0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.23.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-macosx_10_12_x86_64.whl.metadata (6.7 kB)
  Using cached PyPika-0.48.9-py2.py3-

In [1]:
## initial database?
# -> Set to True if you run the notebook for the first time or if you changed the md files
initial_db = False

In [2]:
DATA_PATH = "data/"
OLLAMA_MODEL = "llama2:chat"
OLLAMA_URL = "http://localhost:11434"
CHROMA_PATH = "chroma/"

## langchain split config
# md headers
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

# chunk sizes
chunk_size = 500
chunk_overlap = 100

## Create chunks from md files

In [3]:
from langchain.document_loaders import TextLoader
import os

documents = []

for file in os.listdir(DATA_PATH):
    loader = TextLoader(DATA_PATH + file)
    documents.append(loader.load()[0])

In [4]:
documents[0].metadata
len(documents[0].page_content)

163388

In [5]:
# for doc in documents:
#     print(doc.metadata)

In [6]:
from langchain.text_splitter import MarkdownHeaderTextSplitter


text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
chunks_array= []


for doc in documents:
    chunks_array.append(text_splitter.split_text(doc.page_content))
    # append source metadata to each chunk
    for chunk in chunks_array[-1]:
        # combine metadate
        chunk.metadata = doc.metadata



In [7]:
len(chunks_array)

1

In [8]:
# Char-level splits
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len, add_start_index=True
)


In [9]:
chunks_array_txt_base = []
counter = 0
for document in chunks_array:
    for chunk in document:
        splits = text_splitter.split_documents([chunk])
        chunks_array_txt_base.append(splits)
        counter += 1
        

In [10]:
print(counter)
len(chunks_array_txt_base)

14


14

In [11]:
all_document_chunks = [chunk for document in chunks_array_txt_base for chunk in document]

In [12]:
print(len(all_document_chunks))

414


In [13]:
all_document_chunks[1].page_content

"you will have to check the laws of the country where you are located\nbefore using this eBook.  \n> Title: Alice's Adventures in Wonderland  \n> Author: Lewis Carroll  \n> Release date: June 27, 2008 [eBook #11]\n> Most recently updated: March 30, 2021  \n> Language: English  \n> Credits: Arthur DiBianca and David Widger  \n**_ START OF THE PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND _**"

## Connect to ollama backend

In [17]:
# TEST OLLAMA CONNECTION ##
from langchain_community.llms import Ollama

ollama = Ollama(base_url=OLLAMA_URL, model=OLLAMA_MODEL)

print(ollama("Who is Alice?"))


Alice is a fictional character and the protagonist of Lewis Carroll's children's books "Alice's Adventures in Wonderland" and "Through the Looking-Glass". She is a young girl who falls down a rabbit hole and enters a fantastical world called Wonderland, where she encounters strange creatures and characters, including the Mad Hatter, the Cheshire Cat, and the Queen of Hearts. Alice is known for her curiosity, imagination, and sense of wonder, as well as her ability to navigate the absurd and illogical world of Wonderland with a sense of humor and adventure.


## Create vector db

In [18]:
all_document_chunks[0]

Document(page_content="**The Project Gutenberg eBook of Alice's Adventures in Wonderland**  \n> This ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located", metadata={'source': 'data/AliceImWonderland.md', 'start_index': 0})

In [21]:
# create chroma db or load db from disk
if initial_db:
    from langchain.embeddings import OllamaEmbeddings
    from langchain.vectorstores import Chroma

    chroma_db = Chroma.from_documents(all_document_chunks, OllamaEmbeddings(base_url=OLLAMA_URL, model=OLLAMA_MODEL), persist_directory=CHROMA_PATH)

In [22]:
## load chroma db from disk
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

chroma_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=OllamaEmbeddings(base_url=OLLAMA_URL, model=OLLAMA_MODEL))

In [23]:
# test similarity search
query = "Who is Alice?"

result_docs = chroma_db.similarity_search(query)



In [24]:
for doc in result_docs:
    print(doc)

page_content='“Keep your temper,” said the Caterpillar.  \n“Is that all?” said Alice, swallowing down her anger as well as she\ncould.  \n“No,” said the Caterpillar.  \nAlice thought she might as well wait, as she had nothing else to do,\nand perhaps after all it might tell her something worth hearing. For\nsome minutes it puffed away without speaking, but at last it unfolded\nits arms, took the hookah out of its mouth again, and said, “So you\nthink you’re changed, do you?”' metadata={'source': 'data/AliceImWonderland.md', 'start_index': 2003}
page_content='Will you, won’t you, will you, won’t you, won’t you join the dance?  \n“You can really have no notion how delightful it will be\nWhen they take us up and throw us, with the lobsters, out to sea!”\nBut the snail replied “Too far, too far!” and gave a look askance—\nSaid he thanked the whiting kindly, but he would not join the dance.\nWould not, could not, would not, could not, would not join the dance.\nWould not, could not, would n

## Gradio frontend

In [25]:
from langchain_community.llms import Ollama



def chat_ollama(message, history):
    # initiate ollama
    ollama = Ollama(base_url=OLLAMA_URL, model=OLLAMA_MODEL)

    # search for similar documents in chroma db
    result_chunks = chroma_db.similarity_search(message)
    
    chroma_knowledge = ""
    for id, chunk in enumerate(result_chunks):
        source_id = id + 1
        chroma_knowledge += "[" + str(source_id) +"] \n" + chunk.page_content + "\n"

    sources = ""
    for id, chunk in enumerate(result_chunks):
        source_id = id + 1
        sources += "[" + str(source_id) + "] \n" + chunk.metadata["source"] + "\n"

    prompt = "Answer the following question using the provided knowledge and the chat history:\n\n###KNOWLEDGE: " + chroma_knowledge + "\n###CHAT-HISTORY: " + str(history) + "\n\n###QUESTION: " + message
    result = ollama(prompt) + "\n\n\nReferences:\n" + sources 

    # print(prompt)
    
    return result

In [27]:
chat_ollama("Who is Alice?", "")

'\nBased on the provided knowledge and chat history, Alice is the main character in the story. She is a young girl who has fallen down a rabbit hole and found herself in a strange place filled with unusual creatures.\n\n\nReferences:\n[1] \ndata/AliceImWonderland.md\n[2] \ndata/AliceImWonderland.md\n[3] \ndata/AliceImWonderland.md\n[4] \ndata/AliceImWonderland.md\n'

In [30]:
import gradio as gr
gradio_interface = gr.ChatInterface(
        chat_ollama,
        chatbot=gr.Chatbot(),
        textbox=gr.Textbox(placeholder="Example: Who is Alice?", container=False, scale=7),
        title="The Ollama test chatbot",
        description=f"Ask the {OLLAMA_MODEL} chatbot a question!",
        theme='gradio/base', # themes at https://huggingface.co/spaces/gradio/theme-gallery
        retry_btn=None,
        undo_btn="Delete Previous",
        clear_btn="Clear",

)



## Run app

In [31]:
gradio_interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [32]:
### END OF FILE ###